In [36]:
import os
import time
import pickle
import tensorflow as tf
import keras
from keras import initializers
from keras.layers import Input, Dense, Conv1D, MaxPooling1D
from keras.layers import BatchNormalization
from keras.models import Model
from keras.models import model_from_json
from keras.callbacks import ModelCheckpoint
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
import pandas as pd
import sys
from keras import backend as K
from tensorflow.python.keras.backend import set_session
from time import time
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
# disable GPU
# os.environ["CUDA_DEVICE_ORDER"] = "PIC_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [ ]:
## Set GPU computing
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True # dynamically grow the memory used on the GPU
config.log_device_placement = True 	   # to log device placment (on which device the operation ran)
sess = tf.compat.v1.Session(config=config)
set_session(sess)	# set this Tensorflow session as the default session for keras

In [38]:
# extract 30s centered on PGA
def acc30(records,duration=30,sampling=0.02):
    rec_num,length = records.shape
    length = int(duration/sampling)
    outrecs = np.zeros((rec_num,length),dtype=np.float32)
    for i in range(rec_num):
        irec = records[i,:]
        pga_index = np.argmax(abs(irec))
        if pga_index < length/2:
            rec = irec[:length]
        else:
            rec = irec[(pga_index-int(length/2)):(pga_index+int(length/2)+(length%2 == 1))]
        outrecs[i,:] = rec
    return np.float32(outrecs)


In [39]:
with open("data.pickle","rb") as f:
    tr_data,ts_data = pickle.load(f)

x_train = acc30(tr_data[:,1:])
x_test = acc30(ts_data[:,1:])
y_train = tr_data[:,0].astype(int)-1
y_test = ts_data[:,0].astype(int)-1

In [40]:
history=[]
final=[]
a=np.zeros(100)
b=np.zeros(100)

input_img = keras.Input(shape=(1500,1))
x = Conv1D(64, 3, activation='relu', padding='same',kernel_regularizer='l2')(input_img)
x = MaxPooling1D(2, padding='same')(x)
x = Conv1D(32, 3, activation='relu', padding='same',kernel_regularizer='l2')(x)
x = MaxPooling1D(2, padding='same')(x)
x = Conv1D(16, 3, activation='relu', padding='same',kernel_regularizer='l2')(x)
x = MaxPooling1D(2, padding='same')(x)
x = Conv1D(16, 3, activation='relu', padding='same',kernel_regularizer='l2')(x)
x = MaxPooling1D(2, padding='same')(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dropout(0.3)(x)
x = Dense(64, activation='relu')(x)
category = Dense(3, activation='softmax')(x)

optimizer = 'adam'
model = Model(input_img, category)
model.compile(optimizer= optimizer,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

#checkpoint to save the best only
filepath = 'CPU-cnn1d30-best-up-{}.hdf5'.format(optimizer)
checkpoint = ModelCheckpoint(filepath, monitor="val_loss",verbose=1,save_best_only=True,mode="min")
callbacks_list = [checkpoint]

In [ ]:
# training
import time
from datetime import timedelta
start_time = time.monotonic()
history.append(model.fit(x_train, y_train,
                epochs=50,
                batch_size=10,
                shuffle=True,
                validation_data=(x_test, y_test),
                callbacks=callbacks_list,
                verbose=0))
end_time = time.monotonic()
print(timedelta(seconds=end_time-start_time))
scores = model.evaluate(x_test,y_test,verbose=1)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

In [ ]:
start_time = time.monotonic()
scores = model.evaluate(x_test,y_test,verbose=1)
end_time = time.monotonic()
print(timedelta(seconds=end_time-start_time))

In [ ]:
predicts = model.predict(x_test)
y_pred = np.argmax(predicts,axis=1)
tf.math.confusion_matrix(y_pred,y_test)

In [ ]:
plt.plot(history[0].history['accuracy'])
plt.plot(history[0].history['val_accuracy'])

plt.title('Model Accuracy against Network Size')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['large_train', 'large_test','medium_train','medium_test','small_train','small_test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history[0].history['loss'])
plt.plot(history[0].history['val_loss'])

plt.title('Model Accuracy against Network Size')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['large_train', 'large_test','medium_train','medium_test','small_train','small_test'], loc='upper left')
plt.show()

In [24]:
with open("cnn1d30-data-history.pkl", "wb") as f:
    pickle.dump([y_pred,history[0].history['accuracy'],history[0].history['val_accuracy'],
    history[0].history['loss'],history[0].history['val_loss']],f)